https://huggingface.co/black-forest-labs/FLUX.1-dev|

In [ ]:
#pip install torch

In [ ]:
# !pip install transformers datasets evaluate accelerate

Install Tenserflow

In [ ]:
# pip install tensorflow

pipeline() is the easiest and fastest way to use a pretrained model for inference

https://huggingface.co/docs/transformers/en/main_classes/pipelines

In [ ]:
# !pip install --force-reinstall torch

Topic of the code: ChatBot using a Simple/Small Language Model (SLM)

In [11]:
import random
import nltk
from nltk import bigrams, FreqDist, ConditionalFreqDist

In [12]:
import os
import string

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

For Data

In [15]:
from nltk.corpus import reuters
# Ensure that the NLTK Reuters corpus is downloaded
nltk.download('reuters')

# Create a directory to store the text files
os.makedirs('data', exist_ok=True)

# Iterate through the fileids in the Reuters corpus
for fileid in reuters.fileids():
    # Retrieve the text of the news article
    article_text = ' '.join(reuters.words(fileid))

    # Construct the filename (you can also use
    # the categories in the filename if needed)
    filename = f'data/{fileid.replace("/", "_")}.txt'

    # Write the article text to a file
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(article_text)
        
print("If everything went fine, Reuters articles are saved as text files.")

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


If everything went fine, Reuters articles are saved as text files.


In [16]:
input_data_dir = "data"

In [17]:
# String of punctuation without the full stop
punctuation = string.punctuation.replace('.', '')

In [19]:
def is_hidden(filepath):
    return os.path.basename(filepath).startswith('.')

In [20]:
text_data=""
for filename in os.listdir(input_data_dir):
    filepath = os.path.join(input_data_dir, filename)
    if not is_hidden(filepath):
        with open(filepath) as infile:
            for line in infile:
                if line.strip():  # Check if line is not just whitespace
                    # Remove all punctuation except full stops
                    for char in punctuation:
                        line = line.replace(char, '')
                    text_data += line

In [21]:
len(text_data)

8441834

In [24]:
# nltk.download('punkt_tab')
words = nltk.word_tokenize(text_data.lower())

In [25]:
# Generate bigrams
bi_grams = list(bigrams(words))

In [26]:
# Calculate frequency distribution for each bigram
bi_gram_freq_dist = FreqDist(bi_grams)
print(bi_gram_freq_dist)

<FreqDist with 391847 samples and 1564778 outcomes>


In [29]:
from itertools import islice
# Print the first five elements of the dictionary
first_five_items = list(islice(bi_gram_freq_dist.items(), 5))
for item in first_five_items:
    print(item)

(('asian', 'exporters'), 1)
(('exporters', 'fear'), 2)
(('fear', 'damage'), 1)
(('damage', 'from'), 8)
(('from', 'u'), 27)


In [30]:
# Compute conditional frequency distribution of bigrams
bi_gram_freq = ConditionalFreqDist(bi_grams)

In [31]:
bi_gram_freq['natural']

FreqDist({'gas': 216, 'rubber': 39, 'resources': 9, 'for': 3, 'float': 3, 'disasters': 2, 'that': 2, 'lt': 2, 'lower': 1, 'beverages': 1, ...})

In [32]:
bi_gram_freq['artificial']

FreqDist({'situation': 2, 'barriers': 1, 'sweeteners': 1, 'intelligence': 1, '.': 1, 'controls': 1, 'pace': 1, 'hearts': 1})

In [33]:
import heapq

topk=3
# Create a dictionary to hold the top topk bigrams for each first word
top_bigrams_per_first_word = {}

# Iterate over the bigram frequency distribution
for (first_word, second_word), freq in bi_gram_freq_dist.items():
    # Initialize an empty heap for the first_word if it doesn't exist
    if first_word not in top_bigrams_per_first_word:
        top_bigrams_per_first_word[first_word] = []

    # Add to the heap and maintain top topk
    heapq.heappush(top_bigrams_per_first_word[first_word],
                   (freq, second_word))
    if len(top_bigrams_per_first_word[first_word]) > topk:
        heapq.heappop(top_bigrams_per_first_word[first_word])


In [34]:
top_bigrams_per_first_word['natural']

[(9, 'resources'), (216, 'gas'), (39, 'rubber')]

In [35]:
top_bigrams_per_first_word['artificial']

[(1, 'pace'), (1, 'sweeteners'), (2, 'situation')]

In [36]:
# Convert the heap to a simple list for each first word
for first_word in top_bigrams_per_first_word:
    sorted_bigrams = sorted(
        top_bigrams_per_first_word[first_word], reverse=True)
    top_bigrams_list = []
    for freq, second_word in sorted_bigrams:
        top_bigrams_list.append(second_word)
    top_bigrams_per_first_word[first_word] = top_bigrams_list

# Use these filtered bigrams to create a ConditionalFreqDist
filtered_bi_grams = []
for first_word in top_bigrams_per_first_word:
    for second_word in top_bigrams_per_first_word[first_word]:
        filtered_bi_grams.append((first_word, second_word))

bi_gram_freq = ConditionalFreqDist(filtered_bi_grams)

In [37]:
def generate_sentence(word, num_words):
    word =word.lower()
    for _ in range(num_words):
        print(word, end=' ')
        next_words = [item for item, freq in bi_gram_freq[word].items()]
        if len(next_words) > 0:
            # Randomly choose a next word
            word = random.choice(next_words)
        else:
            break  # Break if the word has no following words
    print()

In [38]:
generate_sentence('Asia', 100)

asia and a new company said . s lt xon unit of its stake to be a year . the u . s stock . the dollar . s stock split . s stock split . the u . 5 mln stg in 1986 net profit of the u trans world s . s stock . the company said it is expected to be made in a year . 5 pct in 1986 net profit 2 mln dlrs . the company said . the dollar . the dollar s lt c nil net profit 2 mln stg in 1986 . 


In [40]:
generate_sentence('bank', 10)

bank of the dollar . the u . s lt 
